<div class="alert alert-block alert-warning">
<b>Note:</b> To run this notebook you have to use 
    <b>Swan</b>. To do so:
    <ul>
        <li>connect to <a href="https://swan004.cern.ch/">https://swan004.cern.ch/</a> </li>
        <li>set 96 Python 3, 4 CPUs, 10 GB RAM, Analytix cluster (<i>suggested configuration</i>) </li>
    </ul>
</div>

# Rucio data access
Rucio data are organised in a distributed environment under the path */project/monitoring/archive/rucio/raw/events/*. We can use **Spark** to handle such kind of datasets in a dataframe-like format.


<div class="alert alert-block alert-info">
    <b>Note:</b> FTS data should be store at <i>/project/monitoring/archive/fts/raw/start</i>  (not sure though: <b>deeper checks required</b>)
</div>

In order to access the data, the first step is to connect to the cluster where they are stored. This is done by creating an instance of the Spark Context class, which links our pc to the master (which then interacts with the workers where data are actually stored and processed).

Once we have a connection, then we have to use a Spark Session as an interface to deal with data in practice:

In [1]:
%%time
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("Issues").getOrCreate()
spark

CPU times: user 51.2 ms, sys: 36.3 ms, total: 87.5 ms
Wall time: 23.4 s


<div class="alert alert-block alert-info">
<b>Note:</b> The getOrCreate() method ensures to create a new Spark Session only if not existing yet, so to avoid problems.
</div>

In [3]:
print("Current (local) path:\n\n")
!pwd

Current (local) path:


/eos/home-l/lclissa/SWAN_projects/transfer_errors


## Exploring cluster
Once we have the Spark session, then we can explore the data available inside the cluster through the attribute *catalog* and its method *listTables()*.

In [4]:
print(spark.catalog.listTables())

[]


<div class="alert alert-block alert-info">
<b>Note:</b> No tables are present in our cluster. This may mean that data are to be retrieved every time the notebook is restarted. Alternatively, tables can store intermediate selections for fast access.
</div>

**Get all data collected in one day**, e.g. 15th August 2019:

In [5]:
%%time

# Rucio data path
path = '/project/monitoring/archive/rucio/raw/events/2019/08/15/*'

# load the data in the json file
res = spark.read.json(path)

CPU times: user 125 ms, sys: 42.3 ms, total: 168 ms
Wall time: 1min 30s


**Get only the first file of one day**

In [6]:
%%time
from termcolor import colored

# Rucio data path
fpath = '/project/monitoring/archive/rucio/raw/events/2019/08/15/'

# get the list of files in the day directory
fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
list_status = fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path(fpath))
files_list = [file.getPath().getName() for file in list_status]

# get the first file's name 
singlefile = files_list[0]

# print filenames
print(colored(singlefile, "red"))
for f in files_list[1:]:
    print(f)

# load the data in the json file
ressin = spark.read.json(fpath+singlefile)

part-00000-407531f7-cddf-4d47-bb95-a95e09ac3a59-c000.json.gz
part-00000-ddb030f3-fd5d-4ecd-9497-b695919417d7-c000.json.gz
part-00000-fd8b224e-dbe7-4603-a0ee-1c41ab1a0a6c-c000.json.gz
part-00001-ddb030f3-fd5d-4ecd-9497-b695919417d7-c000.json.gz
part-00002-ddb030f3-fd5d-4ecd-9497-b695919417d7-c000.json.gz
part-00003-ddb030f3-fd5d-4ecd-9497-b695919417d7-c000.json.gz
part-00004-ddb030f3-fd5d-4ecd-9497-b695919417d7-c000.json.gz
part-00005-ddb030f3-fd5d-4ecd-9497-b695919417d7-c000.json.gz
part-00006-ddb030f3-fd5d-4ecd-9497-b695919417d7-c000.json.gz
part-00007-ddb030f3-fd5d-4ecd-9497-b695919417d7-c000.json.gz
part-00008-ddb030f3-fd5d-4ecd-9497-b695919417d7-c000.json.gz
CPU times: user 8.36 ms, sys: 4.5 ms, total: 12.9 ms
Wall time: 232 ms


In [7]:
print(res.count())
print(ressin.count())

14025866
2507


The first thing we can notice is the number of records: the whole day contains ~14 MLN transfers, while the first file has just 2507 records.

<div class="alert alert-block alert-info">
<b>Note:</b> These are **Rucio** data including both failures and successful transfers.
</div>

However, the first json file is just a part of a file which was fractioned by hadoop/spark. In fact, the filnames testify that there are 2 other *part-00000* files that complete the "first" piece of original data. Let now see the differences:

In [8]:
first_file = spark.read.json(fpath+singlefile[:10]+"*")

In [9]:
print(first_file.count())

2762375


In [10]:
print("Total day columns:", len(res.select('data.*').columns))
print("Total columns first piece of file 0:", len(ressin.select('data.*').columns))
print("Total columns of file 0:", len(first_file.select('data.*').columns))

Total day columns: 59
Total columns first piece of file 0: 23
Total columns of file 0: 59


In [11]:
print(ressin.select('data.*').columns)
print("\n\n", first_file.select('data.*').columns)

['account', 'activity', 'checksum_adler', 'created_at', 'dst_rse', 'dst_url', 'duration', 'event_type', 'name', 'protocol', 'purged_reason', 'reason', 'request_id', 'scope', 'src_rse', 'submitted_at', 'tf_created_at', 'tf_submitted_at', 'tool_id', 'transfer_endpoint', 'transfer_id', 'transfer_link', 'vo']


 ['account', 'activity', 'bytes', 'checksum_adler', 'checksum_md5', 'child_name', 'child_scope', 'child_type', 'childname', 'childscope', 'childtype', 'created_at', 'dataset_name', 'dataset_scope', 'did_type', 'dst_rse', 'dst_rse_id', 'dst_type', 'dst_url', 'duration', 'event_type', 'events', 'expired_at', 'external_host', 'external_id', 'file_size', 'length', 'name', 'previous_request_id', 'progress', 'protocol', 'purged_reason', 'queued_at', 'reason', 'request_id', 'request_type', 'retry_count', 'rse', 'rse_id', 'rule_id', 'scope', 'src_rse', 'src_rse_id', 'src_type', 'src_url', 'started_at', 'state', 'submitted_at', 'tf_created_at', 'tf_started_at', 'tf_submitted_at', 'tf_transfe

Now the number of records is clearly increased (~ 3 MLN). Moreover, we notice that the Hadoop/Spark tool distributes not only rows, but also columns (*23 columns in the first bit VS 59 total columns*). In the next cell we can see which columns are missing.

## Retrieve Errors
In order to get just errors we have to filter the dataframe selecting just *event_type* equal to *transfer-failed* or *deletion-failed*. For simplicity, just consider **file 0**.

In [12]:
%%time
errors = first_file.filter(first_file.data.event_type.isin(['transfer-failed', 'deletion-failed']))
# errors = res.filter(res.data.event_type.isin(['transfer-failed', 'deletion-failed']))
# errors = ressin.filter(ressin.data.event_type.isin(['transfer-failed', 'deletion-failed']))

CPU times: user 5.69 ms, sys: 3.6 ms, total: 9.3 ms
Wall time: 78.2 ms


In [13]:
%%time
errors.count()

CPU times: user 23.2 ms, sys: 8.49 ms, total: 31.6 ms
Wall time: 18.5 s


610200

## Data Exploration
Once we have imported the data, then we can start exploring their structure. The first thing when dealing with pyspark dataframes is to check the schema of the data.

pyspark.sql.dataframe.DataFrame

In [18]:
%%time

print("Data type:",  type(errors), "\n")
errors.printSchema()

Data type: <class 'pyspark.sql.dataframe.DataFrame'> 

root
 |-- data: struct (nullable = true)
 |    |-- account: string (nullable = true)
 |    |-- activity: string (nullable = true)
 |    |-- bytes: long (nullable = true)
 |    |-- checksum_adler: string (nullable = true)
 |    |-- checksum_md5: string (nullable = true)
 |    |-- child_name: string (nullable = true)
 |    |-- child_scope: string (nullable = true)
 |    |-- child_type: string (nullable = true)
 |    |-- childname: string (nullable = true)
 |    |-- childscope: string (nullable = true)
 |    |-- childtype: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- dataset_name: string (nullable = true)
 |    |-- dataset_scope: string (nullable = true)
 |    |-- did_type: string (nullable = true)
 |    |-- dst_rse: string (nullable = true)
 |    |-- dst_rse_id: string (nullable = true)
 |    |-- dst_type: string (nullable = true)
 |    |-- dst_url: string (nullable = true)
 |    |-- duration: dou

In [19]:
errors.columns

['data', 'metadata']

The dataframe **errors** is a multilevel datasets containing both *data* and *metadata* (columns). The latter part is of poor interest for our application, so we can focus just on the data.

The following chunk of code transform the data column into a pyspark dataframe, so to have dataframe utilities available for data exploration.

In [20]:
# data is originally a column
print("First:", type(errors.data))

# then we have to transform it into a dataframe in order to get description functions available
data = errors.select('data.*')
print("Then", type(data))

# show the final structure
print("\nSchema:\n")
data.printSchema()

First: <class 'pyspark.sql.column.Column'>
Then <class 'pyspark.sql.dataframe.DataFrame'>

Schema:

root
 |-- account: string (nullable = true)
 |-- activity: string (nullable = true)
 |-- bytes: long (nullable = true)
 |-- checksum_adler: string (nullable = true)
 |-- checksum_md5: string (nullable = true)
 |-- child_name: string (nullable = true)
 |-- child_scope: string (nullable = true)
 |-- child_type: string (nullable = true)
 |-- childname: string (nullable = true)
 |-- childscope: string (nullable = true)
 |-- childtype: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- dataset_name: string (nullable = true)
 |-- dataset_scope: string (nullable = true)
 |-- did_type: string (nullable = true)
 |-- dst_rse: string (nullable = true)
 |-- dst_rse_id: string (nullable = true)
 |-- dst_type: string (nullable = true)
 |-- dst_url: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- event_type: string (nullable = true)
 |-- events: long (nullable

Let now see how the data look like:

In [21]:
def showDf(df, count=None, percent=None, maxColumns=0):
    if (df == None): return
    import pandas
    from IPython.display import display
    pandas.set_option('display.encoding', 'UTF-8')
    # Pandas dataframe
    dfp = None
    # maxColumns param
    if (maxColumns >= 0):
        if (maxColumns == 0): maxColumns = len(df.columns)
        pandas.set_option('display.max_columns', maxColumns)
    # count param
    if (count == None and percent == None): count = 10 # Default count
    if (count != None):
        count = int(count)
        if (count == 0): count = df.count()
        pandas.set_option('display.max_rows', count)
        dfp = pandas.DataFrame(df.head(count), columns=df.columns)
        display(dfp)
    # percent param
    elif (percent != None):
        percent = float(percent)
        if (percent >=0.0 and percent <= 1.0):
            import datetime
            now = datetime.datetime.now()
#             seed = long(now.strftime("%H%M%S"))
            dfs = df.sample(False, percent)#, seed)
            count = df.count()
            pandas.set_option('display.max_rows', count)
            dfp = dfs.toPandas()    
            display(dfp)
                     
showDf(data, count=6)

,account,activity,bytes,checksum_adler,checksum_md5,child_name,child_scope,child_type,childname,childscope,childtype,created_at,dataset_name,dataset_scope,did_type,dst_rse,dst_rse_id,dst_type,dst_url,duration,event_type,events,expired_at,external_host,external_id,file_size,length,name,previous_request_id,progress,protocol,purged_reason,queued_at,reason,request_id,request_type,retry_count,rse,rse_id,rule_id,scope,src_rse,src_rse_id,src_type,src_url,started_at,state,submitted_at,tf_created_at,tf_started_at,tf_submitted_at,tf_transferred_at,tool_id,transfer_endpoint,transfer_id,transfer_link,transferred_at,url,vo
0,None,Analysis Input,947438342,46fa1375,None,None,None,None,None,None,None,None,None,None,None,FMPHI-UNIBA_DATADISK,None,DISK,srm://lcgdpmse.dnp.fmph.uniba.sk:8446/srm/mana...,1.0,transfer-failed,None,None,None,None,947438342,None,DAOD_TOPQ1.16771848._000045.pool.root.1,2a081ea6f66e4720a7e708bffc06a7c6,None,srm,TRANSFER [2] SOURCE SRM_GET_TURL error on the ...,None,TRANSFER [2] SOURCE SRM_GET_TURL error on the ...,beb79f05de2a49a49ade52d6968145f0,None,None,None,None,None,data18_13TeV,GRIF-LAL_DATADISK,None,DISK,srm://grid05.lal.in2p3.fr:8446/srm/managerv2?S...,2019-08-15 16:10:48,None,2019-08-15 16:10:39,NaN,1.565885e+09,1.565885e+09,1.565885e+09,rucio-conveyor,https://fts3-atlas.cern.ch:8446,b2a1db62-b520-541f-86c2-243d3af905c2,https://fts3-atlas.cern.ch:8449/fts3/ftsmon/#/...,2019-08-15 16:10:49,None,atlas
1,None,None,178763,None,None,None,None,None,None,None,None,2019-08-15 16:23:33,None,None,None,None,None,None,None,NaN,deletion-failed,None,None,None,None,178763,None,user.cohm.mc15_13TeV.361213.Pythia8_Monash_NNP...,None,None,gsiftp,The requested service is not available at the ...,None,The requested service is not available at the ...,None,None,None,NERSC_LOCALGROUPDISK,1eeaf7dcb3264abb857452ff35b5daa4,None,user.cohm,None,None,None,None,None,None,None,1.565886e+09,NaN,NaN,NaN,None,None,None,None,None,gsiftp://dtn01.nersc.gov:2811/projecta/project...,atlas
2,None,Staging,1143508100,34fd7f63,None,None,None,None,None,None,None,None,None,None,None,FZK-LCG2_DATADISK,None,DISK,srm://atlassrm-fzk.gridka.de:8443/srm/managerv...,0.0,transfer-failed,None,None,None,None,1143508100,None,data18_13TeV.00363198.physics_Main.daq.RAW._lb...,None,None,srm,Job has been canceled because it stayed in the...,None,Job has been canceled because it stayed in the...,25187aca8d9446cfa9803304f7406bb1,None,None,None,None,None,data18_13TeV,FZK-LCG2_DATATAPE,None,TAPE,srm://atlassrm-fzk.gridka.de:8443/srm/managerv...,None,None,2019-08-08 12:05:01,NaN,NaN,1.565266e+09,1.565871e+09,rucio-conveyor,https://fts3-atlas.cern.ch:8446,c758178b-d81b-5e81-8836-9045074ce516,https://fts3-atlas.cern.ch:8449/fts3/ftsmon/#/...,2019-08-15 12:14:08,None,atlas
3,None,None,554703918,None,None,None,None,None,None,None,None,2019-08-15 16:23:34,None,None,None,None,None,None,None,NaN,deletion-failed,None,None,None,None,554703918,None,user.cohm.8082835.EXT2._001723.xAOD.pool.root,None,None,gsiftp,The requested service is not available at the ...,None,The requested service is not available at the ...,None,None,None,NERSC_LOCALGROUPDISK,1eeaf7dcb3264abb857452ff35b5daa4,None,user.cohm,None,None,None,None,None,None,None,1.565886e+09,NaN,NaN,NaN,None,None,None,None,None,gsiftp://dtn01.nersc.gov:2811/projecta/project...,atlas
4,None,Staging,2621912416,201c4442,None,None,None,None,None,None,None,None,None,None,None,RAL-LCG2-ECHO_DATADISK,None,DISK,gsiftp://gridftp.echo.stfc.ac.uk:2811/atlas:da...,0.0,transfer-failed,None,None,None,None,2621912416,None,data18_13TeV.00363738.physics_Main.daq.RAW._lb...,None,None,gsiftp,Job has been canceled because it stayed in the...,None,Job has been canceled because it stayed in the...,9e7db44778ee4d48bc26221faceca496,None,None,None,None,None,data18_13TeV,RAL-LCG2_DATATAPE,None,TAPE,srm://srm-atlas.gridpp.rl.ac.uk:8443/srm/manag...,None,None,2019-08-08 12:12:25,NaN,NaN,1.565266e+09,1.565871e+09,rucio-conveyor,https://lcgfts3.gridpp.rl.ac.uk:8

In [22]:
%%time

# 0.00001% of res errors
showDf(data, percent=0.00001)

,account,activity,bytes,checksum_adler,checksum_md5,child_name,child_scope,child_type,childname,childscope,childtype,created_at,dataset_name,dataset_scope,did_type,dst_rse,dst_rse_id,dst_type,dst_url,duration,event_type,events,expired_at,external_host,external_id,file_size,length,name,previous_request_id,progress,protocol,purged_reason,queued_at,reason,request_id,request_type,retry_count,rse,rse_id,rule_id,scope,src_rse,src_rse_id,src_type,src_url,started_at,state,submitted_at,tf_created_at,tf_started_at,tf_submitted_at,tf_transferred_at,tool_id,transfer_endpoint,transfer_id,transfer_link,transferred_at,url,vo
0,None,Staging,2621411200,fabb2443,None,None,None,None,None,None,None,None,None,None,None,BNL-OSG2_DATADISK,None,DISK,srm://dcsrm.usatlas.bnl.gov:8443/srm/managerv2...,0.0,transfer-failed,None,None,None,None,2621411200,None,data18_13TeV.00362345.physics_Main.daq.RAW._lb...,46ad7a4e1c414fd8ba89afbda10b2abd,None,srm,SOURCE [70] srm-ifce err: Communication error ...,None,SOURCE [70] srm-ifce err: Communication error ...,55a8f4c0c9454288a86e734c9e949107,None,None,None,None,None,data18_13TeV,BNL-OSG2_DATATAPE,None,TAPE,srm://dcsrm.usatlas.bnl.gov:8443/srm/managerv2...,2019-08-15 22:04:55,None,2019-08-14 23:19:33,None,1565906695,1565824773,1565906695,rucio-conveyor,https://fts.usatlas.bnl.gov:8446,5af1568e-df57-5e69-9d63-6e032e78ff9a,https://fts.usatlas.bnl.gov:8449/fts3/ftsmon/#...,2019-08-15 22:04:55,None,atlas
1,None,Staging,2621287536,952ef5c5,None,None,None,None,None,None,None,None,None,None,None,BNL-OSG2_DATADISK,None,DISK,srm://dcsrm.usatlas.bnl.gov:8443/srm/managerv2...,69577.0,transfer-failed,None,None,None,None,2621287536,None,data18_13TeV.00352123.physics_Main.daq.RAW._lb...,2cffbf004e0d49b89ac343bf54650551,None,srm,STAGING [70] srm-ifce err: Communication error...,None,STAGING [70] srm-ifce err: Communication error...,ab71a19a7274433da4b542eeec281838,None,None,None,None,None,data18_13TeV,BNL-OSG2_DATATAPE,None,TAPE,srm://dcsrm.usatlas.bnl.gov:8443/srm/managerv2...,2019-08-15 02:42:09,None,2019-08-14 23:22:04,None,1565836929,1565824924,1565906506,rucio-conveyor,https://fts.usatlas.bnl.gov:8446,387a9369-5438-5e1d-8dab-c5c9b9ebd3d3,https://fts.usatlas.bnl.gov:8449/fts3/ftsmon/#...,2019-08-15 22:01:46,None,atlas
2,None,Production Input,917181959,0ee70bf6,None,None,None,None,None,None,None,None,None,None,None,BNL-OSG2_DATADISK,None,DISK,srm://dcsrm.usatlas.bnl.gov:8443/srm/managerv2...,2.0,transfer-failed,None,None,None,None,917181959,None,HITS.18811124._013413.pool.root.1,7483102d6f1a4c058ab67c75ee82569b,None,srm,TRANSFER [70] DESTINATION OVERWRITE srm-ifce e...,None,TRANSFER [70] DESTINATION OVERWRITE srm-ifce e...,44f51dc257154699ab12ba780dd3187e,None,None,None,None,None,mc16_13TeV,CA-SFU-T2_DATADISK,None,DISK,srm://lcg-se1.sfu.computecanada.ca:8443/srm/ma...,2019-08-15 22:05:58,None,2019-08-15 22:05:51,None,1565906758,1565906751,1565906760,rucio-conveyor,https://fts.usatlas.bnl.gov:8446,c33ddf47-7a8d-5d54-88bd-7798e01c626c,https://fts.usatlas.bnl.gov:8449/fts3/ftsmon/#...,2019-08-15 22:06:00,None,atlas
3,None,Staging,2621792552,bcf3363b,None,None,None,None,None,None,None,None,None,None,None,BNL-OSG2_DATADISK,None,DISK,srm://dcsrm.usatlas.bnl.gov:8443/srm/managerv2...,81533.0,transfer-failed,None,None,None,None,2621792552,None,data18_13TeV.00364214.physics_Main.daq.RAW._lb...,619b4624792e41f0a524541028271ccf,None,srm,STAGING [70] srm-ifce err: Communication error...,None,STAGING [70] srm-ifce err: Communication error...,d851ee703a754db28b3ae205eda9e850,None,None,None,None,None,data18_13TeV,BNL-OSG2_DATATAPE,None,TAPE,srm://dcsrm.usatlas.bnl.gov:8443/srm/managerv2...,2019-08-14 23:20:53,None,2019-08-14 15:57:44,None,1565824853,1565798264,1565906386,rucio-conveyor,https://fts.usatlas.bnl.gov:8446,e8b02037-2dd2-550b-8064-69ba64099e0c,https://fts.usatlas.bnl.gov:8449/fts3/ftsmon/#...,2019-08-15 21:59:46,None,atlas


CPU times: user 111 ms, sys: 23.2 ms, total: 135 ms
Wall time: 35.3 s


### Variable summary 

In [24]:
%%time

data_summary = data.describe()

data_summary.toPandas()

CPU times: user 94.8 ms, sys: 30.2 ms, total: 125 ms
Wall time: 1min 12s


,summary,account,activity,bytes,checksum_adler,checksum_md5,child_name,child_scope,child_type,childname,childscope,childtype,created_at,dataset_name,dataset_scope,did_type,dst_rse,dst_rse_id,dst_type,dst_url,duration,event_type,events,expired_at,external_host,external_id,file_size,length,name,...,protocol,purged_reason,queued_at,reason,request_id,request_type,retry_count,rse,rse_id,rule_id,scope,src_rse,src_rse_id,src_type,src_url,started_at,state,submitted_at,tf_created_at,tf_started_at,tf_submitted_at,tf_transferred_at,tool_id,transfer_endpoint,transfer_id,transfer_link,transferred_at,url,vo
0,count,0,340100,610200,340100,2,0,0,0,0,0,0,270100,0,0,0,340100,0,340100,340100,340103,610200,0,0,0,0,610200,0,610200,...,610200,610200,0,610200,340100,0,0,270100,262495,0,610200,340100,0,340100,340100,233951,0,340100,270100,233951,340100,340100,340100,340100,340100,340100,340100,270100,610200
1,mean,None,None,1.3087638981880088E9,Infinity,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,25571.534744153578,None,None,None,None,None,1.3087638981880088E9,None,9.0,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.5658895339391189E9,1.5658485986491745E9,1.5656549416635225E9,1.5658901769800706E9,None,None,None,None,None,None,None
2,stddev,None,None,1.5692908366496181E9,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,33577.213656486005,None,None,None,None,None,1.5692908366496181E9,None,0.0,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,24467.18636822717,92158.46834747924,269723.9757147271,22554.521720958797,None,None,None,None,None,None,None
3,min,None,Analysis Input,0,0000717b,8ccb388c1e4d6d35305c73f888e4b738,None,None,None,None,None,None,2019-08-15 00:00:03,None,None,None,AGLT2_DATADISK,None,DISK,davs://dynafed02.heprc.uvic.ca:8443/atlas/scra...,0.0,deletion-failed,None,None,None,None,0,None,01c4648e-9aed-43ca-b8d4-863b12a0ba37_37390.1.j...,...,davs,DESTINATION [125] [gfal2_cancel] operation can...,None,DESTINATION [125] [gfal2_cancel] operation can...,000034124a1b48818076c93cb86fdb0f,None,None,BNL-OSG2_DATADISK,04368058f6c64909b25c8d622aec1f12,None,data12_8TeV,AGLT2_DATADISK,None,DISK,davs://atlaswebdav-kit.gridka.de:2880/pnfs/gri...,2019-08-08 07:34:33,None,2019-08-07 23:40:34,1565827203,1565249673,1565221234,1565312645,rucio-conveyor,https://fts.usatlas.bnl.gov:8446,0001ed03-2443-590a-a6a2-e34c1f656847,https://fts.usatlas.bnl.gov:8449/fts3/ftsmon/#...,2019-08-09 01:04:05,davs://bohr3226.tier2.hep.manchester.ac.uk:443...,atlas
4,max,None,default,46244014260,fff0a492,c58749d9ad7cb64eba7af6ef4bde0663,None,None,None,None,None,None,2019-08-15 23:59:59,None,None,None,ZA-WITS-CORE_PRODDISK,None,TAPE,srm://wipp-se.weizmann.ac.il:8444/srm/managerv...,85946.0,transfer-failed,None,None,None,None,46244014260,None,user.zzheng.407343.PhPy8EG.DAOD_TOPQ1.e6414_a8...,...,srm,Transfer has been forced-canceled because is h...,None,Transfer has been forced-canceled because is h...,ffffb116a0aa43fa85cc66fc08c60bfe,None,None,ZA-WITS-CORE_SCRATCHDISK,f9eb61c7d0f246ad884c5e07276d3153,None,valid1,ZA-WITS-CORE_PRODDISK,None,TAPE,srm://wipp-se.weizmann.ac.il:8444/srm/managerv...,2019-08-15 23:58:41,None,2019-08-15 23:58:36,1565913599,1565913521,1565913516,1565913539,rucio-conveyor,https://lcgfts3.gridpp.rl.ac.uk:8446,ffff250c-554d-5e15-9e5d-8dc9d606bd37,https://lcgfts3.gridpp.rl.ac.uk:8449/fts3/ftsm...,2019-08-15 23:58:59,srm://svr018.gla.scotgrid.ac.uk:8446/srm/manag...,atlas
